In [199]:
import pandas as pd
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
import pickle
import zlib

In [200]:
DEBUG = False

# OrthoGroups to MSA 
+ load table with the proteins - orthogrous distribution 
+ filter the core orthogroups 
+ load proteins .pkl database
+ get the proteins and the genome positions corresponding to the core OGs

In [201]:
if DEBUG:
    ids = 'phy2_test' #'korea2' #'rso_test' 

    core_ogs = '/Users/devseeva/Desktop/work/rso/OrthoFinder/orthoF_msa_'+ids+'/Species_Tree/Orthogroups_for_concatenated_alignment.txt'
    ogs2prot = '/Users/devseeva/Desktop/work/rso/OrthoFinder/orthoF_msa_'+ids+'/Orthogroups/Orthogroups.tsv'
    prot_db = pd.read_pickle('../'+ids+'_outputs/GENOME_PROTEIN_DB_'+ids+'.pkl')
    path_p_aln = '/Users/devseeva/Desktop/work/rso/OrthoFinder/orthoF_msa_'+ids+'/MultipleSequenceAlignments/'

    msa_out_fasta = '/Users/devseeva/Desktop/work/sm_workflow/snakefiles/'+ids+'_concat.fasta'
    msa_out_phylip = '/Users/devseeva/Desktop/work/sm_workflow/snakefiles/'+ids+'_concat.nexus'
    msa_out_xmfa_path = '/Users/devseeva/Desktop/work/sm_workflow/snakefiles/'+ids+'_concat.xmfa'
    aln2acc = '/Users/devseeva/Desktop/work/sm_workflow/snakefiles/outputs/MSA_ACC_DB_'+ids+'.pkl'
else:
    core_ogs = snakemake.input["core_ogs"]
    ogs2prot = snakemake.input["ogs2prot"]
    prot_db = pd.read_pickle(snakemake.input["prot_db"])
    path_p_aln = snakemake.input["path2aln"]

    msa_out_fasta = snakemake.output["msa_fasta"]
    msa_out_phylip = snakemake.output["msa_phylip"]
    msa_out_xmfa_path = snakemake.output["msa_xmfa"]
    aln2acc = snakemake.output["aln2acc"]

In [202]:
strainsNames2fileNames = {}
for c in prot_db.columns:
    strainsNames2fileNames[c.split('$')[1]] = c.split('$')[0]
strainsNames2fileNames

{'GCF_012562465.1_ASM1256246v1': 'CIAT_078',
 'GCF_001587135.1_ASM158713v1': 'UW163',
 'GCF_002501565.1_ASM250156v1': 'RS_488',
 'GCF_014884745.1_ASM1488474v1': 'UW576',
 'GCF_001587155.1_ASM158715v1': 'IBSBF1503',
 'GCF_000215325.1_ASM21532v1': 'Po82',
 'GCF_002549815.1_ASM254981v1': 'RS_489',
 'GCF_003590605.1_ASM359060v1': 'IBSBF_2571',
 'GCF_001299555.1_ASM129955v1': 'UY031'}

In [203]:
prot_db = prot_db.rename(columns=lambda x: str(x).split('$')[0])
prot_db

,CIAT_078,UW163,RS_488,UW576,IBSBF1503,Po82,RS_489,IBSBF_2571,UY031
OG0000865,{'nucl': b'x\x9c5\x90\xd1\r\x001\x08Bg#|\xb8\x...,{'nucl': b'x\x9c5\x90\xcb\x11\x001\x08Bkc8\xd8...,"{'nucl': b""x\x9c-\x90\xc1\r\x00 \x0c\x02g#<\xb...",{'nucl': b'x\x9c5P\xc7\r\x001\x0c\x9a\r\xf1`\x...,{'nucl': b'x\x9c5\x90\xcb\x11\x001\x08Bkc8\xd8...,{'nucl': b'x\x9c5\x90\xd1\x11\xc00\x08Bg\xf3\x...,{'nucl': b'x\x9c-\x90A\x0eD1\x08B\xcfFX\xb8\xf...,{'nucl': b'x\x9c5\x90\xd1\x11\xc00\x08Bg\xf3\x...,"{'nucl': b""x\x9c-\x90\xc1\r\x00 \x0c\x02g#<\xb..."
OG0001813,{'nucl': b'x\x9c5\x93\xd9\x15\xc40\x08\x03k\xd...,{'nucl': b'x\x9c5\x93\xd9\x15\xc40\x08\x03k\xd...,"{'nucl': b""x\x9c=\x93Q\x12\xc3@\x08B\xcf\xe6\x...",{'nucl': b'x\x9c5\x93\xd1\x15\xc30\x08\x03g\xd...,{'nucl': b'x\x9c5\x93\xd9\x15\xc40\x08\x03k\xd...,{'nucl': b'x\x9c5\x93\xd9\x15\xc40\x08\x03k\xd...,"{'nucl': b""x\x9c=\x93Q\x12\xc3@\x08B\xcf\xe6\x...",{'nucl': b'x\x9c5\x93\xd9\x15\xc40\x08\x03k\xd...,"{'nucl': b""x\x9c=\x93Q\x12\xc3@\x08B\xcf\xe6\x..."
OG0001814,{'nucl': b'x\x9cMP\xdb\r\xc40\x0c\x9a\xcd\xe2\...,{'nucl': b'x\x9cMP\xdb\r\xc40\x0c\x9a\xcd\xe2\...,{'nucl': b'x\x9cMP\xdb\r\xc40\x0c\x9a\r\xf1\xc...,{'nucl': b'x\x9cM\x91\xd1\r\xc40\x0cBg\xb3\xf8...,{'nucl': b'x\x9cMP\xdb\r\xc40\x0c\x9a\xcd\xe2\...,{'nucl': b'x\x9cMP\xdb\r\xc40\x0c\x9a\xcd\xe2\...,{'nucl': b'x\x9c5\x90\xc1\x11\x04A\x08\x02c\xb...,{'nucl': b'x\x9cMP\xdb\r\xc40\x0c\x9a\xcd\xe2\...,{'nucl': b'x\x9cMP\xdb\r\xc40\x0c\x9a\r\xf1\xc...
OG0001815,"{'nucl': b""x\x9c-\x90\xc1\r\x000\x08\x02g#>X\x...","{'nucl': b""x\x9c-\x90\xc1\r\x000\x08\x02g#>X\x...",{'nucl': b'x\x9c-\x90\xc1\r\x000\x08\x02g#>X\x...,{'nucl': b'x\x9c-\x90\xc1\r\x000\x08\x02g#>X\x...,"{'nucl': b""x\x9c-\x90\xc1\r\x000\x08\x02g#>X\x...","{'nucl': b""x\x9c-\x90\xc1\r\x000\x08\x02g#>X\x...","{'nucl': b""x\x9c-\x90\xc1\r\x000\x08\x02g#<X\x...","{'nucl': b""x\x9c-\x90\xc1\r\x000\x08\x02g#>X\x...",{'nucl': b'x\x9c-\x90\xc1\r\x000\x08\x02g#>X\x...
OG0001816,{'nucl': b'x\x9cE\x92\xd1\x91\xc40\x0cBkc\xf8P...,{'nucl': b'x\x9cE\x92\xd1\x91\xc40\x0cBkc\xf8P...,{'nucl': b'x\x9cE\x92\xd9\x11\x031\x0cBkc\xf4\...,{'nucl': b'x\x9cM\x92\xd1\x91\xc40\x0cBkc\xf4\...,{'nucl': b'x\x9cE\x92\xd1\x91\xc40\x0cBkc\xf8P...,{'nucl': b'x\x9cE\x92\xd1\x91\xc40\x0cBkc\xf8P...,"{'nucl': b""x\x9cE\x92Kn\x04A\x0cB\xcf\x86Xx\x1...",{'nucl': b'x\x9cE\x92\xd1\x91\xc40\x0cBkc\xf8P...,{'nucl': b'x\x9cE\x92\xd9\x11\x031\x0cBkc\xf4\...
...,...,...,...,...,...,...,...,...,...
OG0002936,"{'nucl': b""x\x9c=\x91\xd9\x11\x04A\x08Bc\xb3\x...","{'nucl': b""x\x9c=\x91\xd9\x11\x04A\x08Bc\xb3\x...",{'nucl': b'x\x9c=Q\xc9\x11\x04A\x08\x8a\x8d\xe...,{'nucl': b'x\x9c=\x91\xc1\r\x041\x08\x03k\xb3x...,"{'nucl': b""x\x9c=\x91\xd9\x11\x04A\x08Bc\xb3\x...","{'nucl': b""x\x9c=\x91\xd9\x11\x04A\x08Bc\xb3\x...","{'nucl': b""x\x9c5\x91\xcb\x11\x001\x08Bks8\xd8...",{'nucl': b'x\x9c=\x91\xd9\x11\xc3@\x08Ck\xd3\x...,{'nucl': b'x\x9c=Q\xc9\x11\x04A\x08\x8a\x8d\xe...
OG0002935,{'nucl': b'x\x9c=\x92\xdd\r\xc0@\x08\x82g3>\xb...,{'nucl': b'x\x9c=\x92\xdd\r\xc0@\x08\x82g3>\xb...,"{'nucl': b""x\x9cM\x92\xd1\r\x031\x08CgC|x\x01\...",{'nucl': b'x\x9cM\x92\xdb\r\x05A\x08Bk#~\xd8\x...,{'nucl': b'x\x9c=\x92\xdd\r\xc0@\x08\x82g3>\xb...,{'nucl': b'x\x9c=\x92\xdd\r\xc0@\x08\x82g3>\xb...,{'nucl': b'x\x9cM\x92\xdb\r\x05A\x08Bk#~\xd8\x...,{'nucl': b'x\x9c=\x92\xdd\r\xc0@\x08\x82g3>\xb...,"{'nucl': b""x\x9cM\x92\xd1\r\x031\x08CgC|x\x01\..."
OG0002221,{'nucl': b'x\x9c-\x8f\xdb\r\x000\x08\x02g3|\xb...,{'nucl': b'x\x9c-\x8f\xdb\r\x000\x08\x02g3|\xb...,{'nucl': b'x\x9c5\x90\xc1\x11\x00!\x0c\x02kcx\...,{'nucl': b'x\x9c-\x8f\xd1\r\x001\x08Bg#|\xb0\x...,{'nucl': b'x\x9c-\x8f\xdb\r\x000\x08\x02g3|\xb...,{'nucl': b'x\x9c-\x8f\xdb\r\x000\x08\x02g3|\xb...,{'nucl': b'x\x9cU\x8f\xd1\r\xc0@\x08Bg#|\xb0\x...,{'nucl': b'x\x9c-\x8f\xdb\r\x000\x08\x02g3|\xb...,{'nucl': b'x\x9c5\x90\xc1\x11\x00!\x0c\x02kcx\...
OG0002220,{'nucl': b'x\x9c5\x8f\xcb\r\x00A\x08Bk3\x1eh\x...,{'nucl': b'x\x9c5\x8f\xcb\r\x00A\x08Bk3\x1eh\x...,{'nucl': b'x\x9c5\x8f\xdb\r\xc0P\x08Bg#~\xb0\

In [204]:
# get core ogs
f = open(core_ogs,"r")
ogs = []
for l in f:
    ogs.append(l.replace('\n',''))
f.close()
##print(ogs)

In [205]:
# load roteins ids from the OGs table 
# keep only the core OGs

tab_prot = pd.read_csv(ogs2prot, sep='\t', index_col=0)  
tab_prot = tab_prot.loc[ogs]
tab_prot = tab_prot.rename(columns=lambda x: str(x).replace('_protein','')) # ncbi file names correction
tab_prot = tab_prot.rename(columns=strainsNames2fileNames)
tab_prot

,Po82,UY031,UW163,IBSBF1503,RS_488,RS_489,IBSBF_2571,CIAT_078,UW576
Orthogroup,,,,,,,,,
OG0000210,WP_003261125.1,WP_003261125.1,WP_003261125.1,WP_003261125.1,WP_003261125.1,WP_013208914.1,WP_003261125.1,WP_003261125.1,WP_094395590.1
OG0000211,WP_003261138.1,WP_003261138.1,WP_003261138.1,WP_003261138.1,WP_003261138.1,WP_013208922.1,WP_003261138.1,WP_003261138.1,WP_119448024.1
OG0000212,WP_003261153.1,WP_003261153.1,WP_003261153.1,WP_003261153.1,WP_003261153.1,WP_013208932.1,WP_003261153.1,WP_003261153.1,WP_094395604.1
OG0000213,WP_003261159.1,WP_003261159.1,WP_003261159.1,WP_003261159.1,WP_003261159.1,WP_003261159.1,WP_003261159.1,WP_003261159.1,WP_003261159.1
OG0000214,WP_003261220.1,WP_003261220.1,WP_003261220.1,WP_003261220.1,WP_003261220.1,WP_013208611.1,WP_003261220.1,WP_003261220.1,WP_003261220.1
...,...,...,...,...,...,...,...,...,...
OG0003325,WP_184906701.1,WP_020957219.1,WP_014617162.1,WP_014617162.1,WP_003264922.1,WP_043945402.1,WP_174265275.1,WP_014617162.1,WP_003268198.1
OG0003326,WP_184906713.1,WP_079693334.1,WP_080894330.1,WP_080894330.1,WP_079693334.1,WP_081262489.1,WP_174265557.1,WP_080894330.1,WP_168725004.1
OG0003327,WP_184906719.1,WP_020957916.1,WP_042549891.1,WP_042549891.1,WP_020957916.1,WP_057395500.1,WP_057395500.1,WP_042549891.1,WP_094394662.1


In [206]:
genomes_for_og = list(tab_prot.columns) 

#for x in ['pe1', 'pe3', 'pe27', 'pe57', 'pe39']:
#    genomes_for_og.remove(x)
#genomes_for_og

# Get Nucleotide sequences + corresponding positions in genome

# Load single proteins alignments and extract gaps

In [207]:
gaps2tab = pd.DataFrame(columns=tab_prot.columns, index=tab_prot.index)

len_counter = 0

for og in ogs:
    pmsa = path_p_aln + '/' +og + '.fa'
    genome_ids = []
    #if len_counter >= 395140:
      #  print(og, '-----')
        #len_counter = 0
            
    for record in SeqIO.parse(pmsa, "fasta"):  
        #print(pmsa)
        
        header = record.id.split('_')
        
        if record.id.count(header[-2])>1:
            # martinique header structure
            if record.id.startswith(header[-2]):
                genome_id=header[-2]
            else:
                genome_id=header[-3]+'_'+header[-2]
            protein_id = record.id.replace('_'+genome_id,'')
        else:
            # prokka/ncbi genomes header structure 
            protein_id = header[-2]+'_'+header[-1]
            genome_id = strainsNames2fileNames[record.id.replace(protein_id, '').replace('protein_', '')[:-1].replace('_1_', '.1_')]
            #genome_ids.append(genome_id)
            #prot_db.loc[og, genome_id]['og'] = og
        
        #print(pmsa)
        #print(protein_id, genome_id)
        
        # GAPS PROFILE of aa alignment  
        gaps_bool = []
        
        #if og == 'OG0000732' and genome_id == 'to53':
            #print('---', len(record.seq) )
        
        for aa in record.seq:
            if aa == '-':
                gaps_bool.append(1) 
                gaps_bool.append(1) 
                gaps_bool.append(1) 
            else:
                gaps_bool.append(0) 
                gaps_bool.append(0) 
                gaps_bool.append(0)
        
        
        #if genome_id == 'to53':
        #    print(gaps_bool)
        #    print('to53',record.seq)
        #if genome_id == 'to7':
        #    print(gaps_bool)
        #    print('to7', record.seq)
        prot_db.loc[og, genome_id]['aln_gaps'] = gaps_bool
        #prot_db.loc[og, genome_id]['og'] = og
        gaps2tab.loc[og][genome_id] = gaps_bool
        
        #if len(record.seq)*3 != len(prot_db.loc[protein_id, genome_id]['nucl']):
            #print(protein_id, genome_id)
            #print('ffffff',len(record.seq)*3, len(prot_db.loc[protein_id, genome_id]['nucl']))
    #print(og, genome_ids)
gaps2tab

,Po82,UY031,UW163,IBSBF1503,RS_488,RS_489,IBSBF_2571,CIAT_078,UW576
Orthogroup,,,,,,,,,
OG0000210,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
OG0000211,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
OG0000212,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
OG0000213,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
OG0000214,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...,...,...
OG0003325,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, ..."
OG0003326,"[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
OG0003327,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


sss='23-10BR_23-10BR_01627'
sss='CFBP6783_2_CFBP6783_2_02036'
header = sss.split('_')

if sss.count(header[-2])>1:
    if sss.startswith(header[-2]):
        genome_id=header[-2]
    else:
        genome_id=header[-3]+'_'+header[-2]
    protein_id = sss.replace('_'+genome_id,'')
print(protein_id, genome_id)        

import ast
d = ast.literal_eval(prot_db.loc[protein_id, genome_id])
d
d['og'] = og
prot_db.loc[protein_id, genome_id] = d

In [208]:
ogs_to_drop_from_core = []
for g in genomes_for_og:
    for og in ogs:
        
        msa_l = len(prot_db.loc[og, g]['aln_gaps'])
        seq_l = len(zlib.decompress(prot_db.loc[og, g]['nucl']).decode())
        gap_l = gaps2tab.loc[og, g].count(1)
        
        if seq_l + gap_l != msa_l:
            print('Length error in:', g, og, ': MSA length:', msa_l, 'protein length:' ,seq_l)
            ogs_to_drop_from_core.append(og)
            
tab_prot = tab_prot.drop(ogs_to_drop_from_core)
gaps2tab = gaps2tab.drop(ogs_to_drop_from_core)
ogs = [i for i in ogs if i not in ogs_to_drop_from_core]

In [209]:
prot_db=prot_db.loc[ogs]

# Genome-based solution
+ for each genome grab the cols of the core OG table
+ concatenate the corresponding nucleotides from the GENOME_PROTEIN_DB

## TODO length of nucleotides is larger than length of translation*3 ?!

# replace ambiguous bases with gaps!

In [220]:
atcg = {'A', 'C', 'T', 'G'}

for g in genomes_for_og:
    for i in prot_db[g]: 
        nucl = zlib.decompress(i['nucl']).decode()
        ambig = set(nucl).difference(atcg)
        #print(ambig)
        for x in ambig:
            nucl = nucl.replace(x, '-')
        i['nucl'] = zlib.compress(nucl.encode())

# +++ Export MSA

In [221]:
g2nucl = {}
g2pos = {}
g2chr = {}
g2strand = {}
g2acc = {}
g2gaps = {}
g2og = {}


msa_fasta = open(msa_out_fasta, "w")

for g in genomes_for_og:
    print('Genome', g)
    msa_fasta.write('>'+g+'\n')
    
    # get corresponding nucleotide positions 
    g2pos[g] = []
    # get corresponding chr ids 
    g2chr[g] = []
    # get corresponding strands
    g2strand[g] = []
    # get protein accession for each genome position
    #g2acc[g] = []
    # get gaps profile
    g2gaps[g] = []
    # debug
    g2og[g] = []
    
    # get nucleotide sequences for all the core og proteins per genome 
    nucl_per_genome = []
    #print(prot_db.loc[tab_prot[g].values, g])
    
    for i in prot_db[g]: 
        
        ####----nucl_per_genome.append(zlib.decompress(i['nucl']).decode())
        nucl = zlib.decompress(i['nucl']).decode()
        gaps = i['aln_gaps']
        c = 0
        for gap in gaps:
            if gap:
                msa_fasta.write('-')
            else:
                msa_fasta.write(nucl[c])
                c = c + 1
        
        #g2pos[g].extend(i['location'])
        #g2og[g].append(i['og'])
        
        #for counter in range(len(i['aln_gaps'])):
            ###g2chr[g].append(i['chr'])
            ###g2strand[g].append(i['location'].strand)
            #g2acc[g].append(i['acc'])
        
        ###----g2gaps[g].extend(i['aln_gaps'])
    
    msa_fasta.write('\n')
    ###----nucl_per_genome = "".join(nucl_per_genome)
    ###----g2nucl[g] = nucl_per_genome
    
    
    #print('Exported sequence length NO GAPS', len(nucl_per_genome))
    #print('Exported alignment WITH GAPS', len(g2gaps[g]))
    #print('Exported protein accessions', len(g2acc[g]))
    #print('Exported numeric positions', len(g2pos[g]))
    #print('Exported chromosomal positions', len(g2chr[g]))
    #print('Exported strands orientations', len(g2strand[g]))
    #print()
    
    ####### debug 
    #g2nucl[g] = g2nucl[g][:395150]
    #g2gaps[g] = g2gaps[g][:395150]
    #g2acc[g] = g2acc[g][:395150]

msa_fasta.close()
    

Genome Po82
Genome UY031
Genome UW163
Genome IBSBF1503
Genome RS_488
Genome RS_489
Genome IBSBF_2571
Genome CIAT_078
Genome UW576


In [222]:
## msa2ogs instead of g2acc!

msa2ogs = []
for og in prot_db.index:
    i = prot_db.loc[og][g]
    for counter in range(len(i['aln_gaps'])):
        msa2ogs.append(og)

# MSA repeats problem 

some proteins sequence of the OrthoFinder may have long repeat sequences, which are not present in the orthologs. In the resulting MSA produced by OrthoFinder the repeated sequences are skipped:

e.g. 
+ original input: <br>
\> p1 <br>
ABCXYZ <br>
\> p2 <br>
ABCABCXYZ <br>
+ MSA output:<br>
\> p1 <br>
ABCXYZ <br>
\> p2 <br>
ABCXYZ <br>

This results in large shifts in the final nucleotides MSA <br>

Examples from to53

OG: OG0000732; gene: DEOEPAP_01560
MNSKTDRPAEGAGATADRAAREAAVARALECLADHAGDHQTLTGEPLVSHARGTVAILEGLRVDAPSLQAAALFLLPILATDNEKAIEPAFGGEVARLVHDVRQLLRIGAIAGMASPAEAGVTRKNEAEARRAQVEALRKMLLAFAQDIRVVLIRLASRLQSLRWLAQNKLPAPEGMARETLDIYAPLANRLGIWQLKWELEDLGFRFEDPDTYKRIARLLDEKRIEREQFIGEAIARLQQTLRDAGIHAEVSGRPKHIYSIWRKMRGKDLDFADLYDVRAFRVIVDDIKDCYTVLGFVHHMWQPIPKEFDDYISRPKTNGYKSLHTVNSKTDRPAEGAGATADRAAREAAVARALECLADHAGDHQTLTGEPLVSHARGTVAILEGLRVDAPSLQAAALFLLPILATDNEKAIEPAFGGEVARLVHDVRQLLRIGAIAGMASPAEAGVTRKNEAEARRAQVEALRKMLLAFAQDIRVVLIRLASRLQSLRWLAQNKLPAPEGMARETLDIYAPLANRLGIWQLKWELEDLGFRFEDPDTYKRIARLLDEKRIEREQFIGEAIARLQQTLRDAGIHAEVSGRPKHIYSIWRKMRGKDLDFADLYDVRAFRVIVDDIKDCYTVLGFVHHMWQPIPKEFDDYISRPKTNGYKSLHTVVIGDDGRALEVQIRTREMHHFAEYGVAAHWRYKEAGSKGYAGQFSASERYDEKIAWLRQLLAWKDDAEHTVAQEDSPWEQLKHTELDDHIYVLTPQARVIALPQGATPVDFAYYLHSDLGHRCRGARVDGTMVPLNTPLKNGQTVEIVTVKQGGPSRDWLNPELHYLASNRARAKVRAWFNALELEETLAQGRILIDKTLQREGKTAVNLEELAAKLGFKTPDELYAMVAKDEFSLRHVEAVLRTDGAPPAPADDEASITKKSRATSVARGAKSGVLVVGVDSLLTQMSRCCKPAPPDLIVGFVTRGRGVSIHRQNCATFQQLAARAPGRVIQTEWGNRGEAVYPVDIHVEALDRQGLLRDISEILSREKINVTGVRTLSSKGVAKMQFTAEVSEATQLQRALILIEEVTGVLSARRK
VNSKTDRPAEGAGATADRAAREAAVARALECLADHAGDHQTLTGEPLVSHARGTVAILEGLRVDAPSLQAAALFLLPILATDNEKAIEPAFGGEVARLVHDVRQLLRIGAIAGMASPAEAGVTRKNEAEARRAQVEALRKMLLAFAQDIRVVLIRLASRLQSLRWLAQNKLPAPEGMARETLDIYAPLANRLGIWQLKWELEDLGFRFEDPDTYKRIARLLDEKRIEREQFIGEAIARLQQTLRDAGIHAEVSGRPKHIYSIWRKMRGKDLDFADLYDVRAFRVIVDDIKDCYTVLGFVHHMWQPIPKEFDDYISRPKTNGYKSLHT                                                                                                                                                                                                                                                                                                                                       VVIGDDGRALEVQIRTREMHHFAEYGVAAHWRYKEAGSKGYAGQFSASERYDEKIAWLRQLLAWKDDAEHTVAQEDSPWEQLKHTELDDHIYVLTPQARVIALPQGATPVDFAYYLHSDLGHRCRGARVDGTMVPLNTPLKNGQTVEIVTVKQGGPSRDWLNPELHYLASNRARAKVRAWFNALELEETLAQGRILIDKTLQREGKTAVNLEELAAKLGFKTPDELYAMVAKDEFSLRHVEAVLRTDGAPPAPADDEASITKKSRATSVARGAKSGVLVVGVDSLLTQMSRCCKPAPPDLIVGFVTRGRGVSIHRQNCATFQQLAARAPGRVIQTEWGNRGEAVYPVDIHVEALDRQGLLRDISEILSREKINVTGVRTLSSKGVAKMQFTAEVSEATQLQRALILIEEVTGVLSARRK

OG: OG0002212; gene: GDEOEPAP_02088
MTRVDLRSAKPDYDGPLLRLDDLSVRFEAEHNEWIEAVRHVSFDLHAGERFALVGESGSGKTVTALSIMRLLADAHYSGRILFEGRDLLAASERQMRGLRGADVAMVFQEPMTALNPLYTIGNQIVETLELHEGLDKRAAKARAIALLEHTGIAEAPRRFDAFPHQLSGGQRQRAMIAMALACSPKLLVADEPTTALDVTVRMQILQLLRDLQAEYGMAVMLITHDLNMVRAFAERVGVMERGVLVETGETATVFSAPQHPYTVRLLESRPQRDVLPLVPLAPVLLEADQLTVTYDRHRPGFAGWFRADPFTAVDQVSLQLREGETLGIVGESGSGKTTLAQTLLGLQRPKAGDLRFLGSSLLGQTRDERRAVRARMQVVFQDPFGSLSPRMTIEQIVGEGLALHQPHVNDAERRQRVIEALREVGLDRTALGRYPHEFSGGQRQRIAIARVLILKPQVLVLDEPTSALDVSIQQQVLSLLSALQKKYNLSYLFISHDLAVIRAMSHRVMVMKDGKVVEAGDTEAVLASPSHPYTCKLMAAAALKPLQPLQPPG
MTRVDLRSAKPDYDGPLLRLDDLSVRFEAEHNEWIEAVRHVSFDLHAGERFALVGESGSGKTVTALSIMRLLADAHYSGRILFEGRDLLAASERQMRGLRGADVAMVFQEPMTALNPLYTIGNQIVETLELHEGLDKRAAKARAIALLEHTGIAEAPRRFDAFPHQLSGGQRQRAMIAMALACSPKLLVADEPTTALDVTVRMQILQLLRDLQAEYGMAVMLITHDLNMVRAFAERVGVMERGVLVETGETATVFSAPQHPYTVRLLESRPQRDVLPLVPLAPVLLEADQLTVTYDRHRPGFAGWFRADPFTAVDQVSLQLREGETLGIVGESGSGKTTLAQTLLGLQRPKAGDLRFLGSSLLGQTRDERRAVRARMQVVFQDPFGSLSPRMTIEQIVGEGLALHQPHVNDAERRQRVIEALREVGLDRTALGRYPHEFSGGQRQRIAIARVLILKPQVLVLDEPTSALDVSIQQQVLSLLSALQKKYNLSYLFISHDLAVIRAMSHRVMVMKDGKVVEAGDTEAVLASPSHPYTCKLMAAAALKPLQ   PPG

OG: OG0003692; gene: GDEOEPAP_04673
MRHFPTLHSVALIPSGAAITRAAVTGHARHPRAPVRNALGGTVLLGTFAAAIPAFAAPAGGTVTSGSGSIGQNGTTTTITQTSSRLAIDWSAFGIQAGETVNFIQPGAGAIALNRVTGHEATSIFGSLNANGTVFILNPNGVLFGGGAQVNVGGLVASTLGLSNADFEAGRYALSGGSTAGVTNQGTITVPSGGKVALIANSVDNAGNISAPRGSVLLAGAGNVTLTLADGSPLGYTISAGAARTLVNNGGMIVADGGRVVLTARGLDSLSESVVNTTGVVRARTVGNNQGTIELIGDPMAGLTQVSGQIDASAPDGGGNGGSVKVLGTKVGVFDGARIDVSGMAGGGTALIGGNAQGAGPEPNATATYVAPTAAIDASAIRQGNGGRLIVWGTDVANVHGSLSANGGAQGGNGGHVETSGHALDTDGIAVSVAGGGGG  TGGLWLLDPYNVTLSAGTQTGGAFSGNVWTPSASGSLVNTSSIQSLLNSGGNVTITTTGAGTQEGNIAINGSISKTAGGNASLTLIADGRITTNASSGTHRTITSTSGTLDVSMTARATTSAASTSGINLRYVDINANGGNISATASGAQSGTAAALSLQNSSWSTTGAGNISLSGTLPDNGNSQGVFLNANTLTTASGSITVSGTSGGIATITGTNSNGAPLFSTSNIGVYLAGGNTLRSTGGGAIALTGTATGATATWAGSGVTLSALDSLSTSGAVTISGTASNPVSSTYRNQLSTVNIAGTGSNAASLTGGTVTITGTNSVVGNASSTSNGNAAVKLDGKVNLTATAGPINIAGSNAGGDGVWGSGSGAVTMSAPASSSITISARSLDSVSGYSGFYIGGGSATLTFATAAPVSIAAESLVGARKAFWNKGGLTVPGNLSIVTTGGAIADDTTHGGYFHVTGSTSIDSSGAGNTVSLTNSGNTFAGAISLNAGDTTLASSTSLTLDSSTVNGALNLTAPGITQSGAVSAAGTATLNGGNAAITLTNDGNAFNALAVTGAAGVNVVNANALSVTGVTSTGAVRFTTLSGDLATRGNWSISGGDLTLSAGAGNTRGTASGGNVDSQATLAVDAGRTITVYSGAVGSTVLGGTLAGRAAAGSGNFRYNRQDGDAPGADGIGDGATYVMYRERPTVTVTPTDAANTKVYDGGGASDPALAYTIVGQVNGDSATQILSGSLARAAGQNAGRYAISQGTLADQLGYQTVLDVGHAFSITPAPLLITANGASRTAGQDNPAFSATFAGFVGGQTAQTADLRGTLRLLTSADAGSPAGSYAIVPDGISSNNYSIRYAAGTLSVLPASPAPMVAVPGEPSYMNAVIDMQARIRAAGLASAPEPAERSATGWRTTTATFRQDDTASDNVPGNRLAAQE
MRHFPTLHSVALIPSGAAITRAAVTGHARHPRAPVRNALGGTVLLGTFAAAIPAFAAPAGGTVTSGSGSIGQNGTTTTITQTSSRLAIDWSAFGIQAGETVNFIQPGAGAIALNRVTGHEATSIFGSLNANGTVFILNPNGVLFGGGAQVNVGGLVASTLGLSNADFEAGRYALSGGSTAGVTNQGTITVPSGGKVALIANSVDNAGNISAPRGSVLLAGAGNVTLTLADGSPLGYTISAGAARTLVNNGGMIVADGGRVVLTARGLDSLSESVVNTTGVVRARTVGNNQGTIELIGDPMAGLTQVSGQIDASAPDGGGNGGSVKVLGTKVGVFDGARIDVSGMAGGGTALIGGNAQGAGPEPNATATYVAPTAAIDASAIRQGNGGRLIVWGTDVANVHGSLSANGGAQGGNGGHVETSGHALDTDGIAVSVAGGGGGGGTGGLWLLDPYNVTLSAGTQTGGAFSGNVWTPSASGSLVNTSSIQSLLNSGGNVTITTTGAGTQEGNIAINGSISKTAGGNASLTLIADGRITTNASSGTHRTITSTSGTLDVSMTARATTSAASTSGINLRYVDINANGGNISATASGAQSGTAAALSLQNSSWSTTGAGNISLSGTLPDNGNSQGVFLNANTLTTASGSITVSGTSGGIATITGTNSNGAPLFSTSNIGVYLAGGNTLRSTGGGAIALTGTATGATATWAGSGVTLSALDSLSTSGAVTISGTASNPVSSTYRNQLSTVNIAGTGSNAASLTGGTVTITGTNSVVGNASSTSNGNAAVKLDGKVNLTATAGPINIAGSNAGGDGVWGSGSGAVTMSAPASSSITISARSLDSVSGYSGFYIGGGSATLTFATAAPVSIAAESLVGARKAFWNKGGLTVPGNLSIVTTGGAIADDTTHGGYFHVTGSTSIDSSGAGNTVSLTNSGNTFAGAISLNAGDTTLASSTSLTLDSSTVNGALNLTAPGITQSGAVSAAGTATLNGGNAAITLTNDGNAFNALAVTGAAGVNVVNANALSVTGVTSTGAVRFTTLSGDLATRGNWSISGGDLTLSAGAGNTRGTASGGNVDSQATLAVDAGRTITVYSGAVGSTVLGGTLAGRAAAGSGNFRYNRQDGDAPGADGIGDGATYVMYRERPTVTVTPTDAANTKVYDGGGASDPALAYTIVGQVNGDSATQILSGSLARAAGQNAGRYAISQGTLADQLGYQTVLDVGHAFSITPAPLLITANGASRTAGQDNPAFSATFAGFVGGQTAQTADLRGTLRLLTSADAGSPAGSYAIVPDGISSNNYSIRYAAGTLSVLPASPAPMVAVPGEPSYMNAVIDMQARIRAAGLASAPEPAERSATGWRTTTATFRQDDTASDNVPGNRLAAQE

In [223]:
import difflib

s_nucl ="MTRVDLRSAKPDYDGPLLRLDDLSVRFEAEHNEWIEAVRHVSFDLHAGERFALVGESGSGKTVTALSIMRLLADAHYSGRILFEGRDLLAASERQMRGLRGADVAMVFQEPMTALNPLYTIGNQIVETLELHEGLDKRAAKARAIALLEHTGIAEAPRRFDAFPHQLSGGQRQRAMIAMALACSPKLLVADEPTTALDVTVRMQILQLLRDLQAEYGMAVMLITHDLNMVRAFAERVGVMERGVLVETGETATVFSAPQHPYTVRLLESRPQRDVLPLVPLAPVLLEADQLTVTYDRHRPGFAGWFRADPFTAVDQVSLQLREGETLGIVGESGSGKTTLAQTLLGLQRPKAGDLRFLGSSLLGQTRDERRAVRARMQVVFQDPFGSLSPRMTIEQIVGEGLALHQPHVNDAERRQRVIEALREVGLDRTALGRYPHEFSGGQRQRIAIARVLILKPQVLVLDEPTSALDVSIQQQVLSLLSALQKKYNLSYLFISHDLAVIRAMSHRVMVMKDGKVVEAGDTEAVLASPSHPYTCKLMAAAALKPLQPLQPPG"
s_msa = "MTRVDLRSAKPDYDGPLLRLDDLSVRFEAEHNEWIEAVRHVSFDLHAGERFALVGESGSGKTVTALSIMRLLADAHYSGRILFEGRDLLAASERQMRGLRGADVAMVFQEPMTALNPLYTIGNQIVETLELHEGLDKRAAKARAIALLEHTGIAEAPRRFDAFPHQLSGGQRQRAMIAMALACSPKLLVADEPTTALDVTVRMQILQLLRDLQAEYGMAVMLITHDLNMVRAFAERVGVMERGVLVETGETATVFSAPQHPYTVRLLESRPQRDVLPLVPLAPVLLEADQLTVTYDRHRPGFAGWFRADPFTAVDQVSLQLREGETLGIVGESGSGKTTLAQTLLGLQRPKAGDLRFLGSSLLGQTRDERRAVRARMQVVFQDPFGSLSPRMTIEQIVGEGLALHQPHVNDAERRQRVIEALREVGLDRTALGRYPHEFSGGQRQRIAIARVLILKPQVLVLDEPTSALDVSIQQQVLSLLSALQKKYNLSYLFISHDLAVIRAMSHRVMVMKDGKVVEAGDTEAVLASPSHPYTCKLMAAAALKPLQPPG"

s_msa = "MRHFPTLHSVALIPSGAAITRAAVTGHARHPRAPVRNALGGTVLLGTFAAAIPAFAAPAGGTVTSGSGSIGQNGTTTTITQTSSRLAIDWSAFGIQAGETVNFIQPGAGAIALNRVTGHEATSIFGSLNANGTVFILNPNGVLFGGGAQVNVGGLVASTLGLSNADFEAGRYALSGGSTAGVTNQGTITVPSGGKVALIANSVDNAGNISAPRGSVLLAGAGNVTLTLADGSPLGYTISAGAARTLVNNGGMIVADGGRVVLTARGLDSLSESVVNTTGVVRARTVGNNQGTIELIGDPMAGLTQVSGQIDASAPDGGGNGGSVKVLGTKVGVFDGARIDVSGMAGGGTALIGGNAQGAGPEPNATATYVAPTAAIDASAIRQGNGGRLIVWGTDVANVHGSLSANGGAQGGNGGHVETSGHALDTDGIAVSVAGGGGGTGGLWLLDPYNVTLSAGTQTGGAFSGNVWTPSASGSLVNTSSIQSLLNSGGNVTITTTGAGTQEGNIAINGSISKTAGGNASLTLIADGRITTNASSGTHRTITSTSGTLDVSMTARATTSAASTSGINLRYVDINANGGNISATASGAQSGTAAALSLQNSSWSTTGAGNISLSGTLPDNGNSQGVFLNANTLTTASGSITVSGTSGGIATITGTNSNGAPLFSTSNIGVYLAGGNTLRSTGGGAIALTGTATGATATWAGSGVTLSALDSLSTSGAVTISGTASNPVSSTYRNQLSTVNIAGTGSNAASLTGGTVTITGTNSVVGNASSTSNGNAAVKLDGKVNLTATAGPINIAGSNAGGDGVWGSGSGAVTMSAPASSSITISARSLDSVSGYSGFYIGGGSATLTFATAAPVSIAAESLVGARKAFWNKGGLTVPGNLSIVTTGGAIADDTTHGGYFHVTGSTSIDSSGAGNTVSLTNSGNTFAGAISLNAGDTTLASSTSLTLDSSTVNGALNLTAPGITQSGAVSAAGTATLNGGNAAITLTNDGNAFNALAVTGAAGVNVVNANALSVTGVTSTGAVRFTTLSGDLATRGNWSISGGDLTLSAGAGNTRGTASGGNVDSQATLAVDAGRTITVYSGAVGSTVLGGTLAGRAAAGSGNFRYNRQDGDAPGADGIGDGATYVMYRERPTVTVTPTDAANTKVYDGGGASDPALAYTIVGQVNGDSATQILSGSLARAAGQNAGRYAISQGTLADQLGYQTVLDVGHAFSITPAPLLITANGASRTAGQDNPAFSATFAGFVGGQTAQTADLRGTLRLLTSADAGSPAGSYAIVPDGISSNNYSIRYAAGTLSVLPASPAPMVAVPGEPSYMNAVIDMQARIRAAGLASAPEPAERSATGWRTTTATFRQDDTASDNVPGNRLAAQE"
s_nucl =  "MRHFPTLHSVALIPSGAAITRAAVTGHARHPRAPVRNALGGTVLLGTFAAAIPAFAAPAGGTVTSGSGSIGQNGTTTTITQTSSRLAIDWSAFGIQAGETVNFIQPGAGAIALNRVTGHEATSIFGSLNANGTVFILNPNGVLFGGGAQVNVGGLVASTLGLSNADFEAGRYALSGGSTAGVTNQGTITVPSGGKVALIANSVDNAGNISAPRGSVLLAGAGNVTLTLADGSPLGYTISAGAARTLVNNGGMIVADGGRVVLTARGLDSLSESVVNTTGVVRARTVGNNQGTIELIGDPMAGLTQVSGQIDASAPDGGGNGGSVKVLGTKVGVFDGARIDVSGMAGGGTALIGGNAQGAGPEPNATATYVAPTAAIDASAIRQGNGGRLIVWGTDVANVHGSLSANGGAQGGNGGHVETSGHALDTDGIAVSVAGGGGGGGTGGLWLLDPYNVTLSAGTQTGGAFSGNVWTPSASGSLVNTSSIQSLLNSGGNVTITTTGAGTQEGNIAINGSISKTAGGNASLTLIADGRITTNASSGTHRTITSTSGTLDVSMTARATTSAASTSGINLRYVDINANGGNISATASGAQSGTAAALSLQNSSWSTTGAGNISLSGTLPDNGNSQGVFLNANTLTTASGSITVSGTSGGIATITGTNSNGAPLFSTSNIGVYLAGGNTLRSTGGGAIALTGTATGATATWAGSGVTLSALDSLSTSGAVTISGTASNPVSSTYRNQLSTVNIAGTGSNAASLTGGTVTITGTNSVVGNASSTSNGNAAVKLDGKVNLTATAGPINIAGSNAGGDGVWGSGSGAVTMSAPASSSITISARSLDSVSGYSGFYIGGGSATLTFATAAPVSIAAESLVGARKAFWNKGGLTVPGNLSIVTTGGAIADDTTHGGYFHVTGSTSIDSSGAGNTVSLTNSGNTFAGAISLNAGDTTLASSTSLTLDSSTVNGALNLTAPGITQSGAVSAAGTATLNGGNAAITLTNDGNAFNALAVTGAAGVNVVNANALSVTGVTSTGAVRFTTLSGDLATRGNWSISGGDLTLSAGAGNTRGTASGGNVDSQATLAVDAGRTITVYSGAVGSTVLGGTLAGRAAAGSGNFRYNRQDGDAPGADGIGDGATYVMYRERPTVTVTPTDAANTKVYDGGGASDPALAYTIVGQVNGDSATQILSGSLARAAGQNAGRYAISQGTLADQLGYQTVLDVGHAFSITPAPLLITANGASRTAGQDNPAFSATFAGFVGGQTAQTADLRGTLRLLTSADAGSPAGSYAIVPDGISSNNYSIRYAAGTLSVLPASPAPMVAVPGEPSYMNAVIDMQARIRAAGLASAPEPAERSATGWRTTTATFRQDDTASDNVPGNRLAAQE"

s_nucl ="MNSKTDRPAEGAGATADRAAREAAVARALECLADHAGDHQTLTGEPLVSHARGTVAILEGLRVDAPSLQAAALFLLPILATDNEKAIEPAFGGEVARLVHDVRQLLRIGAIAGMASPAEAGVTRKNEAEARRAQVEALRKMLLAFAQDIRVVLIRLASRLQSLRWLAQNKLPAPEGMARETLDIYAPLANRLGIWQLKWELEDLGFRFEDPDTYKRIARLLDEKRIEREQFIGEAIARLQQTLRDAGIHAEVSGRPKHIYSIWRKMRGKDLDFADLYDVRAFRVIVDDIKDCYTVLGFVHHMWQPIPKEFDDYISRPKTNGYKSLHTVNSKTDRPAEGAGATADRAAREAAVARALECLADHAGDHQTLTGEPLVSHARGTVAILEGLRVDAPSLQAAALFLLPILATDNEKAIEPAFGGEVARLVHDVRQLLRIGAIAGMASPAEAGVTRKNEAEARRAQVEALRKMLLAFAQDIRVVLIRLASRLQSLRWLAQNKLPAPEGMARETLDIYAPLANRLGIWQLKWELEDLGFRFEDPDTYKRIARLLDEKRIEREQFIGEAIARLQQTLRDAGIHAEVSGRPKHIYSIWRKMRGKDLDFADLYDVRAFRVIVDDIKDCYTVLGFVHHMWQPIPKEFDDYISRPKTNGYKSLHTVVIGDDGRALEVQIRTREMHHFAEYGVAAHWRYKEAGSKGYAGQFSASERYDEKIAWLRQLLAWKDDAEHTVAQEDSPWEQLKHTELDDHIYVLTPQARVIALPQGATPVDFAYYLHSDLGHRCRGARVDGTMVPLNTPLKNGQTVEIVTVKQGGPSRDWLNPELHYLASNRARAKVRAWFNALELEETLAQGRILIDKTLQREGKTAVNLEELAAKLGFKTPDELYAMVAKDEFSLRHVEAVLRTDGAPPAPADDEASITKKSRATSVARGAKSGVLVVGVDSLLTQMSRCCKPAPPDLIVGFVTRGRGVSIHRQNCATFQQLAARAPGRVIQTEWGNRGEAVYPVDIHVEALDRQGLLRDISEILSREKINVTGVRTLSSKGVAKMQFTAEVSEATQLQRALILIEEVTGVLSARRK"
s_msa = "VNSKTDRPAEGAGATADRAAREAAVARALECLADHAGDHQTLTGEPLVSHARGTVAILEGLRVDAPSLQAAALFLLPILATDNEKAIEPAFGGEVARLVHDVRQLLRIGAIAGMASPAEAGVTRKNEAEARRAQVEALRKMLLAFAQDIRVVLIRLASRLQSLRWLAQNKLPAPEGMARETLDIYAPLANRLGIWQLKWELEDLGFRFEDPDTYKRIARLLDEKRIEREQFIGEAIARLQQTLRDAGIHAEVSGRPKHIYSIWRKMRGKDLDFADLYDVRAFRVIVDDIKDCYTVLGFVHHMWQPIPKEFDDYISRPKTNGYKSLHTVVIGDDGRALEVQIRTREMHHFAEYGVAAHWRYKEAGSKGYAGQFSASERYDEKIAWLRQLLAWKDDAEHTVAQEDSPWEQLKHTELDDHIYVLTPQARVIALPQGATPVDFAYYLHSDLGHRCRGARVDGTMVPLNTPLKNGQTVEIVTVKQGGPSRDWLNPELHYLASNRARAKVRAWFNALELEETLAQGRILIDKTLQREGKTAVNLEELAAKLGFKTPDELYAMVAKDEFSLRHVEAVLRTDGAPPAPADDEASITKKSRATSVARGAKSGVLVVGVDSLLTQMSRCCKPAPPDLIVGFVTRGRGVSIHRQNCATFQQLAARAPGRVIQTEWGNRGEAVYPVDIHVEALDRQGLLRDISEILSREKINVTGVRTLSSKGVAKMQFTAEVSEATQLQRALILIEEVTGVLSARRK"

#s1-s2
output_list = [li for li in difflib.ndiff(s_nucl, s_msa) if li[0] != ' ']
print(output_list)

repeat_seq = ''.join([i[-1] for i in output_list if i[0] == '-'])
print('rep:', repeat_seq)

# count diff in nucl string 
print(s_nucl.count(repeat_seq[1:]))
print(s_msa.count(repeat_seq[1:]))

['- M', '- N', '- S', '- K', '- T', '- D', '- R', '- P', '- A', '- E', '- G', '- A', '- G', '- A', '- T', '- A', '- D', '- R', '- A', '- A', '- R', '- E', '- A', '- A', '- V', '- A', '- R', '- A', '- L', '- E', '- C', '- L', '- A', '- D', '- H', '- A', '- G', '- D', '- H', '- Q', '- T', '- L', '- T', '- G', '- E', '- P', '- L', '- V', '- S', '- H', '- A', '- R', '- G', '- T', '- V', '- A', '- I', '- L', '- E', '- G', '- L', '- R', '- V', '- D', '- A', '- P', '- S', '- L', '- Q', '- A', '- A', '- A', '- L', '- F', '- L', '- L', '- P', '- I', '- L', '- A', '- T', '- D', '- N', '- E', '- K', '- A', '- I', '- E', '- P', '- A', '- F', '- G', '- G', '- E', '- V', '- A', '- R', '- L', '- V', '- H', '- D', '- V', '- R', '- Q', '- L', '- L', '- R', '- I', '- G', '- A', '- I', '- A', '- G', '- M', '- A', '- S', '- P', '- A', '- E', '- A', '- G', '- V', '- T', '- R', '- K', '- N', '- E', '- A', '- E', '- A', '- R', '- R', '- A', '- Q', '- V', '- E', '- A', '- L', '- R', '- K', '- M', '- L', '- L'

In [224]:
# SAVE PROTEIN ACC FOR MAPPING

with open(aln2acc, 'wb') as handle:
    pickle.dump(msa2ogs, handle, protocol=pickle.HIGHEST_PROTOCOL)


# replace ambiguous bases with gaps!

for g in g2nucl:
    atcg = {'A', 'C', 'T', 'G'}
    ambig = set(g2nucl[g]).difference(atcg)
    
    print("before", set(g2nucl[g]))
    for b in ambig:
        g2nucl[g] = g2nucl[g].replace(b, '-')
    print("after ",set(g2nucl[g]))

In [225]:
#str(prot_db.loc["WP_097388881.1","GCF_002501565.1_ASM250156v1_protein"]['nucl'])
#for v in prot_db['GCF_001587135.1_ASM158713v1_protein'].values:
#    print(v)

# Export MSA

msa_fasta = open(msa_out_fasta, "w")
for g in g2nucl:
    print(g, len(g2nucl[g]))
    msa_fasta.write('>'+g+'\n') ##+g.split('_')[2][:10]+'\n')
    
    i = 0
    for gap in g2gaps[g]:
        if gap:
            msa_fasta.write('-')
        else:
            msa_fasta.write(g2nucl[g][i])
            i = i + 1
    msa_fasta.write('\n')
msa_fasta.close()

# Convert to Phylip

#SeqIO.convert(msa_out_fasta, "fasta", msa_out_phylip, "nexus")

nexus = open(msa_out_phylip, 'w')
nexus.write('#NEXUS\n')
nexus.write('begin data;\n')
nexus.write('dimensions ntax='+str(len(genomes_for_og))+' nchar='+str(len(g2gaps[g]))+' ;\n')
nexus.write('format datatype=dna ;\n')
nexus.write('matrix\n')

for record in SeqIO.parse(msa_out_fasta, "fasta"):
    nexus.write(record.description+'\t'+str(record.seq)+'\n')
    
nexus.write(';\n')
nexus.write('end;\n')

nexus.close()
#SeqIO.write(annotated_records, msa_out_phylip, "nexus")


# Export XMFA

In [226]:
msa_out_xmfa = open(msa_out_xmfa_path, 'w')

for og in ogs:
    pmsa = path_p_aln + '/' +og + '.fa'
    for record in SeqIO.parse(pmsa, "fasta"):
        #print(record.id)
        #g_id = record.id.split('_WP')[0].replace('_1_', '.1_')
        #p_id = record.id.split('_protein_')[-1]
        # prokka changes 
        ##g_id = record.id.split('_')[0]
        ##p_id = record.id.replace(record.id.split('_')[0]+'_', '')
        
        header = record.id.split('_')
        
        if record.id.count(header[-2])>1:
            # martinique header structure
            if record.id.startswith(header[-2]):
                g_id=header[-2]
            else:
                g_id=header[-3]+'_'+header[-2]
            p_id = record.id.replace('_'+g_id,'')
        else:
            # prokka/ncbi genomes header structure 
            p_id = header[-2]+'_'+header[-1]
            g_id = strainsNames2fileNames[record.id.replace(p_id, '').replace('protein_', '')[:-1].replace('_1_', '.1_')]
                                           
        nns = zlib.decompress(prot_db.loc[og, g_id]['nucl']).decode() 
        #msa_out_xmfa.write('>'+record.id.split('_')[3][:10]+'\n')
        #prokka
        msa_out_xmfa.write('>'+g_id+'\n')
        
        atcg = {'A', 'T', 'G', 'C','-'}
        ambig = set(nns).difference(atcg)
        #print(ambig)
        for b in ambig:
            nns = str(nns).replace(b, '-')
        #print('aaaaaa',  nns)
        
        gaps_v = gaps2tab.loc[og, g_id]
        i = 0
        for gap in gaps_v:
            if gap:
                msa_out_xmfa.write('-')
            else:
                msa_out_xmfa.write(nns[i])
                i = i + 1
        msa_out_xmfa.write('\n')
    msa_out_xmfa.write('=' + '\n')
msa_out_xmfa.close()

# Test the position mapping
+ grab sequences from the original gb files using g2pos, g2strand and g2chr
+ load exported core OG nucleotide fasta file
+ compare the sequences 

(note: reverse compliment!)

In [227]:
if DEBUG:
    # for the compliment strand mapping
    from Bio.Seq import Seq
    Seq("A").complement()

In [228]:
if DEBUG:
    import os
    gbks = '/Users/devseeva/Desktop/work/sm_workflow/geneTrack/inputs/panX/'+ids
    ccc=0

    test_fasta = {}
    for entry in os.scandir(gbks):
        #if ccc>0:
        #    break
        ccc = ccc+1
        if (entry.path.endswith(".gbk") and entry.is_file()):
            genome = os.path.basename(entry).replace('.gbk', '').replace('_genomic', '')
            genome = strainsNames2fileNames[genome]
            test_fasta[genome] = []
            print('Genome:', genome)

            # map chr id to the sequence
            chr_seq = {}
            for record in SeqIO.parse(entry.path, "gb"):
                print('Chr',record.id, 'with', len(record.features), 'features')
                chr_seq[record.id] = record.seq

            # get corresponding mapping (from above)
            mapping_chr = g2chr[genome]
            mapping_pos = g2pos[genome]
            mapping_str = g2strand[genome]
            
            print(g2chr[genome][:10])
            print(g2pos[genome][:10])
            print(g2strand[genome][:10])
            
            print(len(mapping_chr), len(mapping_pos))
            assert(len(mapping_chr) == len(mapping_pos))

            # iterate through the original positions 
            for i in range(len(mapping_chr)):
                if mapping_str[i] == 1:
                    test_fasta[genome].append(chr_seq[mapping_chr[i]][mapping_pos[i]])
                else:
                    compl = str(Seq(chr_seq[mapping_chr[i]][mapping_pos[i]]).complement())
                    test_fasta[genome].append(compl)
            test_fasta[genome] = "".join(test_fasta[genome])
            print("Core nucl exported:", len(test_fasta[genome]))

            print()
            

Genome: CIAT_078
Chr NZ_CP051296.1 with 6609 features
Chr NZ_CP051295.1 with 3093 features
[]
[]
[]
0 0
Core nucl exported: 0

Genome: UW163
Chr NZ_CP012939.1 with 6609 features
Chr NZ_CP012941.1 with 197 features
Chr NZ_CP012942.1 with 91 features
Chr NZ_CP012940.1 with 3077 features
[]
[]
[]
0 0
Core nucl exported: 0

Genome: RS_488
Chr NZ_CP021652.1 with 6416 features
Chr NZ_CP021653.1 with 3175 features
[]
[]
[]
0 0
Core nucl exported: 0

Genome: UW576
Chr NZ_CP051175.1 with 6698 features
Chr NZ_CP051176.1 with 2934 features
[]
[]
[]
0 0
Core nucl exported: 0

Genome: IBSBF1503
Chr NZ_CP012943.1 with 6474 features
Chr NZ_CP012944.1 with 3242 features
[]
[]
[]
0 0
Core nucl exported: 0

Genome: Po82
Chr NC_017574.1 with 6548 features
Chr NC_017575.1 with 3143 features
[]
[]
[]
0 0
Core nucl exported: 0

Genome: RS_489
Chr NZ_CP021766.1 with 6414 features
Chr NZ_CP021767.1 with 3151 features
[]
[]
[]
0 0
Core nucl exported: 0

Genome: IBSBF_2571
Chr NZ_CP026307.1 with 6546 features
C

In [229]:
if DEBUG:

    for record in SeqIO.parse(msa_out_fasta, "fasta"):
        print(record.id, len(record.seq))
        print('test length', len(record.seq))
        print('Test:', test_fasta[record.id] == str(record.seq))
        print()


Po82 2987190
test length 2987190
Test: False

UY031 2987190
test length 2987190
Test: False

UW163 2987190
test length 2987190
Test: False

IBSBF1503 2987190
test length 2987190
Test: False

RS_488 2987190
test length 2987190
Test: False

RS_489 2987190
test length 2987190
Test: False

IBSBF_2571 2987190
test length 2987190
Test: False

CIAT_078 2987190
test length 2987190
Test: False

UW576 2987190
test length 2987190
Test: False



f = "/Users/devseeva/Desktop/work/pan-genome-visualization/data/ncbi-rso/metainfo.tsv"
x = pd.read_csv(f, sep='\t')  
phys = []
for a in x["accession"]:
    phys.append(a.split(".1_")[1][:10])
x["phy_id"] = phys
x.to_csv("/Users/devseeva/Desktop/work/sm_workflow/snakefiles/inputs/phy4_test_metadata/phy4_test_cleaned_data.csv", index=False)